# Get summary GA

In [66]:
import pandas as pd
from pathlib import Path
from collections import defaultdict
from tabulate import tabulate
from rdkit import Chem
import shutil

from operator import itemgetter

from support_mvp.backup_plot_diagram.plot_diagram_schrock import (
    HARTREE2KCAL,
    base,
    collection,
    dicts,
    get_diagram_dicts,
    get_diagram_dicts_return_gibbs
)

In [67]:

smiles = ['CCCN1CCCC(N)(C2CCCC2)C1','CCCOC(=O)NCC1(N)CCCCC1', 'NC1(CCCCCl)CCCCC1', 'CC(C)(N)CCC1CCCCC1', 'CC(N)Cc1ccc(Cc2ccccc2)cc1', 'CC(N)Cc1ccc(CCl)cc1', 'CC(=O)Nc1ccc(F)c(C(=O)Cc2ccnc(N)c2)c1', 'C=C(C)CC(C)(C)N', 'CC(=O)c1ccc(F)c(C(=O)CN)c1', 'CC(=O)c1ccc(F)c(C(=O)N=CN)c1', 'CC(=O)c1cc(O)c(C(=O)CN)cc1O', 'N#Cc1ccnc(C#N)c1N', 'N#Cc1ccnc(C(=O)Cl)c1N', 'N#Cc1cnc(C#N)c(N)c1', 'N#Cc1ncc(CC(=O)O)cc1N', 'N#Cc1ncc(C(=O)CO)cc1N', 'Nc1ccccc1N=CC(=O)Cl', 'NCCc1ncnc2ccccc12', 'NCC=Cc1ncnc2ccccc12', 'NC=NC(=O)c1cccc(Br)c1', 'NCCCc1ncnc2ccccc12', 'NCCCOc1ccnc2cccnc12', 'NCCCOc1ncnc2cccnc12', 'NCCOC(=O)c1ncnc2ccccc12']

pkls = [
"/home/magstr/Documents/GB_GA/notebooks/df_nh3ton2.pkl",
"/home/magstr/Documents/GB_GA/notebooks/df_n2binding.pkl",
"/home/magstr/Documents/GB_GA/notebooks/df_reduction_step.pkl",
]

base = Path("/home/magstr/Documents/")
path_dir = base / "schrock/diagrams_schrock/dft/b3lyp_candidates"

In [68]:
# Canidates i aready have singlepoints for:
already_sp = ['C=C(C)CC(C)(C)N', 'N#Cc1ncc(C(=O)CO)cc1N', 'NCCc1ncnc2ccccc12', 'NCCCOc1ccnc2cccnc12', 'CC(N)Cc1ccc(Cc2ccccc2)cc1', 'CC(C)(N)CCC1CCCCC1', 'CC(N)Cc1ccc(CCl)cc1', 'N#Cc1cnc(C#N)c(N)c1', 'N#Cc1ccnc(C#N)c1N', 'N#Cc1ccnc(C(=O)Cl)c1N', 'CCCOC(=O)NCC1(N)CCCCC1','NCCOC(=O)c1ncnc2ccccc12']

In [69]:
for_dft = list(set(smiles) - set(already_sp))
print(for_dft)

['CC(=O)c1cc(O)c(C(=O)CN)cc1O', 'CC(=O)c1ccc(F)c(C(=O)N=CN)c1', 'NC1(CCCCCl)CCCCC1', 'NCCCc1ncnc2ccccc12', 'Nc1ccccc1N=CC(=O)Cl', 'CC(=O)Nc1ccc(F)c(C(=O)Cc2ccnc(N)c2)c1', 'CCCN1CCCC(N)(C2CCCC2)C1', 'NCC=Cc1ncnc2ccccc12', 'N#Cc1ncc(CC(=O)O)cc1N', 'NCCCOc1ncnc2cccnc12', 'CC(=O)c1ccc(F)c(C(=O)CN)c1', 'NC=NC(=O)c1cccc(Br)c1']


## Create singlepoint folders for the remaining 12 for_dft

## Create summary dict

In [70]:
options_ref = {
                "exchange": True,
                "thermal_correction": True,
                'diagrams': [
                    [
                        base / "schrock/diagrams_schrock/dft/b3lyp_sp",
                        "orca.out",
                        "FINAL SINGLE",
                        dicts["parts_orca_b3lyp_sp"],
                        Path("/home/magstr/Documents/schrock/diagrams_schrock/vibrations_original_xtb/")
                    ],
                ]
            }
    

options_ref = defaultdict(lambda: False, options_ref)
compare_data_ref, delta_gibbs_ref = get_diagram_dicts_return_gibbs(options_ref)

Doing diagram /home/magstr/Documents/schrock/diagrams_schrock/dft/b3lyp_sp
NH3->N2 : 8.48247880116105
NH3->N2_NH3 : -2.072646460495889
NH3+->NH3 : 1.5236350516788661
lol


In [71]:
d3 = {}
summary_dict = {}
for elem in smiles:
    
    print(elem)
    m = Chem.MolFromSmiles(elem)
    # Replace tricky primary amines in the frag:
    prim_match = Chem.MolFromSmarts("[NX3;H2]")
    rm = Chem.ReplaceSubstructs(
                                m,
                                prim_match,
                                Chem.MolFromSmiles("[1*]"),
                                replaceAll=True,
                            )[0]
    new_smiles=Chem.MolToSmiles(rm)
    
    for f in pkls:
        mol_df = pd.read_pickle(f)
    # Get the scoring
        tmp = mol_df.loc[mol_df["smiles"] == elem]
        if not tmp.empty:
            summary_dict[new_smiles] = {'scoring function':tmp["scoring_function"].values[0]}
            print("scoring", tmp["scoring_function"].values[0])
            break
        else:
            val = ""
            continue

    folders = sorted(path_dir.rglob(f"*{elem}"))
    options = {
                "exchange": True,
                "thermal_correction": True,
                'diagrams': [
                    [
                        folders[0],
                        "orca.out",
                        "FINAL SINGLE",
                        dicts["parts_orca_b3lyp_sp"],
                        Path(f'/home/magstr/Documents/schrock/diagrams_schrock/dft/b3lyp_candidates/xtb_vibrations/{elem}')
                    ],
                ]
            }
    options = defaultdict(lambda: False, options)
    try: 
        compare_data, delta_gibbs = get_diagram_dicts_return_gibbs(options)
    except Exception as e:
        print(e)
        summary_dict.pop(new_smiles)
        print(f"PROBLEM: {elem}")
        continue
    
    options_ref = {
                "exchange": True,
                "thermal_correction": True,
                'diagrams': [
                    [
                        base / "schrock/diagrams_schrock/dft/b3lyp_sp",
                        "orca.out",
                        "FINAL SINGLE",
                        dicts["parts_orca_b3lyp_sp"],
                        Path("/home/magstr/Documents/schrock/diagrams_schrock/vibrations_original_xtb/")
                    ],
                ]
            }
    

    options_ref = defaultdict(lambda: False, options_ref)
    compare_data_ref, delta_gibbs_ref = get_diagram_dicts_return_gibbs(options_ref)

    summary_dict[new_smiles]['NH3toN2'] = compare_data[elem]['Mo_N2_exchange']-compare_data[elem]['Mo_NH3']
    summary_dict[new_smiles]['NH3toN2_NH3'] = compare_data[elem]['Mo_N2_NH3']-compare_data[elem]['Mo_NH3']
    summary_dict[new_smiles]['NH3+toNH3'] = compare_data[elem]['Mo_NH3']-compare_data[elem]['Mo_NH3+']
    
    
    delta_gibbs_ref.pop('Mo_N2_exchange')
    delta_gibbs_ref.pop('Mo_N2_bare')
    delta_gibbs_ref.pop('Mo')
    
    delta_gibbs.pop('Mo_N2_exchange')
    delta_gibbs.pop('Mo_N2_bare')
    delta_gibbs.pop('Mo')
    
    
    # Find step that increased most compared to refference
    d3 = {key: delta_gibbs[key] - delta_gibbs_ref.get(key, 0) for key in delta_gibbs}
    summary_dict[new_smiles]['smiles'] = elem
    summary_dict[new_smiles]['max_change'] = d3[max(d3, key=d3.get)]

CCCN1CCCC(N)(C2CCCC2)C1
scoring NH3 -> N2
Doing diagram /home/magstr/Documents/schrock/diagrams_schrock/dft/b3lyp_candidates/CCCN1CCCC(N)(C2CCCC2)C1
negative seek position -60096
PROBLEM: CCCN1CCCC(N)(C2CCCC2)C1
CCCOC(=O)NCC1(N)CCCCC1
scoring NH3 -> N2
Doing diagram /home/magstr/Documents/schrock/diagrams_schrock/dft/b3lyp_candidates/CCCOC(=O)NCC1(N)CCCCC1
negative seek position -63445
PROBLEM: CCCOC(=O)NCC1(N)CCCCC1
NC1(CCCCCl)CCCCC1
scoring NH3 -> N2
Doing diagram /home/magstr/Documents/schrock/diagrams_schrock/dft/b3lyp_candidates/NC1(CCCCCl)CCCCC1
NH3->N2 : -11.953359075821936
NH3->N2_NH3 : 5.458954010158777
NH3+->NH3 : 9.43881183071062
lol
Doing diagram /home/magstr/Documents/schrock/diagrams_schrock/dft/b3lyp_sp
NH3->N2 : 8.48247880116105
NH3->N2_NH3 : -2.072646460495889
NH3+->NH3 : 1.5236350516788661
lol
CC(C)(N)CCC1CCCCC1
scoring NH3 -> N2
Doing diagram /home/magstr/Documents/schrock/diagrams_schrock/dft/b3lyp_candidates/CC(C)(N)CCC1CCCCC1
NH3->N2 : -14.088762069121003
NH3->N2_

In [72]:
summary_dict

{'[1*]C1(CCCCCl)CCCCC1': {'scoring function': 'NH3 -> N2',
  'NH3toN2': -11.842641026934075,
  'NH3toN2_NH3': 18.36421815472886,
  'NH3+toNH3': 7.948534999005062,
  'smiles': 'NC1(CCCCCl)CCCCC1',
  'max_change': 12.053837862648834},
 '[1*]C(C)(C)CCC1CCCCC1': {'scoring function': 'NH3 -> N2',
  'NH3toN2': -12.570386861327108,
  'NH3toN2_NH3': 13.954394368324529,
  'NH3+toNH3': 8.686312332086288,
  'smiles': 'CC(C)(N)CCC1CCCCC1',
  'max_change': 10.097566883539912},
 '[1*]C(C)Cc1ccc(Cc2ccccc2)cc1': {'scoring function': 'NH3 -> N2',
  'NH3toN2': -12.32232766112918,
  'NH3toN2_NH3': 16.36899883709225,
  'NH3+toNH3': 11.784682079672336,
  'smiles': 'CC(N)Cc1ccc(Cc2ccccc2)cc1',
  'max_change': 13.950689647964737},
 '[1*]C(C)Cc1ccc(CCl)cc1': {'scoring function': 'NH3 -> N2',
  'NH3toN2': -11.437449086340507,
  'NH3toN2_NH3': 14.924803070521762,
  'NH3+toNH3': 10.01301215566042,
  'smiles': 'CC(N)Cc1ccc(CCl)cc1',
  'max_change': 17.714460188260205},
 '[1*]C(C)(C)CC(=C)C': {'scoring function': 

In [73]:
mapping = {'NH3 -> N2': 'NH3toN2', 'NH3 -> NH3 - N2': 'NH3toN2_NH3', 'NH3+ -> NH3':'NH3+toNH3'}

In [74]:
def create_latex_tables(summary_dict):

    # Example code:
    # https://colab.research.google.com/drive/1Iq10lHznMngg1-Uoo-QtpTPii1JDYSQA?usp=sharing#scrollTo=K7NNR1Vg40Vo

    # Simple header
    headers = ["Smiles",'smiles2',"Scoring", "\Delta E", 'Max \delta\delta E']
    
    rows = [ ]
    for k, v in summary_dict.items():
        rows.append([k,v['smiles'],v['scoring function'],v[mapping[v['scoring function']]], v['max_change']])

    # Sort rough 
    rows = sorted(rows, key=itemgetter(3))

    print("Tabulate Table:")
    print(tabulate(rows, headers=headers))

    # Formatting option: floatfmt=".3f"
    print("\nTabulate Latex:")
    print(tabulate(rows, headers=headers, tablefmt="latex", floatfmt=".2f"))

    return

In [75]:
create_latex_tables(summary_dict)

Tabulate Table:
Smiles                          smiles2                      Scoring            \Delta E    Max \delta\delta E
------------------------------  ---------------------------  ---------------  ----------  --------------------
[1*]c1ccccc1N=CC(=O)Cl          Nc1ccccc1N=CC(=O)Cl          NH3+ -> NH3       -27.2843                50.0978
[1*]c1cc(C#N)cnc1C#N            N#Cc1cnc(C#N)c(N)c1          NH3+ -> NH3       -23.3859                35.2067
[1*]C(C)(C)CCC1CCCCC1           CC(C)(N)CCC1CCCCC1           NH3 -> N2         -12.5704                10.0976
[1*]c1c(C#N)ccnc1C(=O)Cl        N#Cc1ccnc(C(=O)Cl)c1N        NH3+ -> NH3       -12.4275                20.3572
[1*]C(C)Cc1ccc(Cc2ccccc2)cc1    CC(N)Cc1ccc(Cc2ccccc2)cc1    NH3 -> N2         -12.3223                13.9507
[1*]C1(CCCCCl)CCCCC1            NC1(CCCCCl)CCCCC1            NH3 -> N2         -11.8426                12.0538
[1*]C(C)Cc1ccc(CCl)cc1          CC(N)Cc1ccc(CCl)cc1          NH3 -> N2         -11.4374         

## No thermal corrections

In [88]:
d3 = {}
summary_dict = {}
for elem in smiles:
    
    print(elem)
    m = Chem.MolFromSmiles(elem)
    # Replace tricky primary amines in the frag:
    prim_match = Chem.MolFromSmarts("[NX3;H2]")
    rm = Chem.ReplaceSubstructs(
                                m,
                                prim_match,
                                Chem.MolFromSmiles("[1*]"),
                                replaceAll=True,
                            )[0]
    new_smiles=Chem.MolToSmiles(rm)
    
    for f in pkls:
        mol_df = pd.read_pickle(f)
    # Get the scoring
        tmp = mol_df.loc[mol_df["smiles"] == elem]
        if not tmp.empty:
            summary_dict[new_smiles] = {'scoring function':tmp["scoring_function"].values[0]}
            print("scoring", tmp["scoring_function"].values[0])
            break
        else:
            val = ""
            continue

    folders = sorted(path_dir.rglob(f"*{elem}"))
    options = {
                "exchange": True,
                "thermal_correction": False,
                'diagrams': [
                    [
                        folders[0],
                        "orca.out",
                        "FINAL SINGLE",
                        dicts["parts_orca_b3lyp_sp"],
                        Path(f'/home/magstr/Documents/schrock/diagrams_schrock/dft/b3lyp_candidates/xtb_vibrations/{elem}')
                    ],
                ]
            }
    options = defaultdict(lambda: False, options)
    try: 
        compare_data, delta_gibbs = get_diagram_dicts_return_gibbs(options)
    except Exception as e:
        print(e)
        summary_dict.pop(new_smiles)
        print(f"PROBLEM: {elem}")
        continue
    
    options_ref = {
                "exchange": True,
                "thermal_correction": False,
                'diagrams': [
                    [
                        base / "schrock/diagrams_schrock/dft/b3lyp_sp",
                        "orca.out",
                        "FINAL SINGLE",
                        dicts["parts_orca_b3lyp_sp"],
                        Path("/home/magstr/Documents/schrock/diagrams_schrock/vibrations_original_xtb/")
                    ],
                ]
            }
    

    options_ref = defaultdict(lambda: False, options_ref)
    compare_data_ref, delta_gibbs_ref = get_diagram_dicts_return_gibbs(options_ref)
    
    
    summary_dict[new_smiles]['NH3toN2'] = compare_data[elem]['Mo_N2_exchange']-compare_data[elem]['Mo_NH3']
    summary_dict[new_smiles]['NH3toN2_NH3'] = compare_data[elem]['Mo_N2_NH3']-compare_data[elem]['Mo_NH3']
    summary_dict[new_smiles]['NH3+toNH3'] = compare_data[elem]['Mo_NH3']-compare_data[elem]['Mo_NH3+']
    
    
    delta_gibbs_ref.pop('Mo_N2_exchange')
    delta_gibbs_ref.pop('Mo_N2_bare')
    delta_gibbs_ref.pop('Mo')
    
    delta_gibbs.pop('Mo_N2_exchange')
    delta_gibbs.pop('Mo_N2_bare')
    delta_gibbs.pop('Mo')
    
    
    # Find step that increased most compared to refference
    d3 = {key: delta_gibbs[key] - delta_gibbs_ref.get(key, 0) for key in delta_gibbs}
    summary_dict[new_smiles]['max_change'] = d3[max(d3, key=d3.get)]

CCCN1CCCC(N)(C2CCCC2)C1
scoring NH3 -> N2
Doing diagram /home/magstr/Documents/schrock/diagrams_schrock/dft/b3lyp_candidates/CCCN1CCCC(N)(C2CCCC2)C1
negative seek position -60096
PROBLEM: CCCN1CCCC(N)(C2CCCC2)C1
CCCOC(=O)NCC1(N)CCCCC1
scoring NH3 -> N2
Doing diagram /home/magstr/Documents/schrock/diagrams_schrock/dft/b3lyp_candidates/CCCOC(=O)NCC1(N)CCCCC1
negative seek position -63445
PROBLEM: CCCOC(=O)NCC1(N)CCCCC1
NC1(CCCCCl)CCCCC1
scoring NH3 -> N2
Doing diagram /home/magstr/Documents/schrock/diagrams_schrock/dft/b3lyp_candidates/NC1(CCCCCl)CCCCC1
NH3->N2 : -11.953359075821936
NH3->N2_NH3 : 5.458954010158777
NH3+->NH3 : 9.43881183071062
Doing diagram /home/magstr/Documents/schrock/diagrams_schrock/dft/b3lyp_sp
NH3->N2 : 8.48247880116105
NH3->N2_NH3 : -2.072646460495889
NH3+->NH3 : 1.5236350516788661
CC(C)(N)CCC1CCCCC1
scoring NH3 -> N2
Doing diagram /home/magstr/Documents/schrock/diagrams_schrock/dft/b3lyp_candidates/CC(C)(N)CCC1CCCCC1
NH3->N2 : -14.088762069121003
NH3->N2_NH3 : 1.

In [89]:
for k, v in summary_dict.items():
        print(k,v)

[1*]C1(CCCCCl)CCCCC1 {'scoring function': 'NH3 -> N2', 'NH3toN2': -11.953359075821936, 'NH3toN2_NH3': 5.458954010158777, 'NH3+toNH3': 9.43881183071062, 'max_change': 11.547267994843423}
[1*]C(C)(C)CCC1CCCCC1 {'scoring function': 'NH3 -> N2', 'NH3toN2': -14.088762069121003, 'NH3toN2_NH3': 1.8013190906494856, 'NH3+toNH3': 10.126163286622614, 'max_change': 9.500804516486824}
[1*]C(C)Cc1ccc(Cc2ccccc2)cc1 {'scoring function': 'NH3 -> N2', 'NH3toN2': -13.84582714131102, 'NH3toN2_NH3': 6.152772002853453, 'NH3+toNH3': 12.423572551459074, 'max_change': 13.50305961072445}
[1*]C(C)Cc1ccc(CCl)cc1 {'scoring function': 'NH3 -> N2', 'NH3toN2': -11.433736371342093, 'NH3toN2_NH3': 3.560693414416164, 'NH3+toNH3': 11.974166839383543, 'max_change': 17.065927658230066}
[1*]C(C)(C)CC(=C)C {'scoring function': 'NH3 -> N2', 'NH3toN2': -9.27844085264951, 'NH3toN2_NH3': 3.490633698645979, 'NH3+toNH3': 9.894694034010172, 'max_change': 15.280527591705322}
[1*]CC(=O)c1cc(C(C)=O)ccc1F {'scoring function': 'NH3 -> N

In [90]:
mapping = {'NH3 -> N2': 'NH3toN2', 'NH3 -> NH3 - N2': 'NH3toN2_NH3', 'NH3+ -> NH3':'NH3+toNH3'}

In [91]:
rows =[]
for k, v in summary_dict.items():
    rows.append([k,v['scoring function'],v[mapping[v['scoring function']]], v['max_change']])

In [92]:
def create_latex_tables():

    # Example code:
    # https://colab.research.google.com/drive/1Iq10lHznMngg1-Uoo-QtpTPii1JDYSQA?usp=sharing#scrollTo=K7NNR1Vg40Vo

    # Simple header
    headers = ["Smiles","Scoring", "\Delta E", 'Max \delta\delta E']
    
    rows = [ ]
    for k, v in summary_dict.items():
        rows.append([k,v['scoring function'],v[mapping[v['scoring function']]], v['max_change']])

    # Sort rough 
    rows = sorted(rows, key=itemgetter(2))

    print("Tabulate Table:")
    print(tabulate(rows, headers=headers))

    # Formatting option: floatfmt=".3f"
    print("\nTabulate Latex:")
    print(tabulate(rows, headers=headers, tablefmt="latex", floatfmt=".2f"))

    return

In [93]:
create_latex_tables()

Tabulate Table:
Smiles                          Scoring             \Delta E    Max \delta\delta E
------------------------------  ---------------  -----------  --------------------
[1*]c1ccccc1N=CC(=O)Cl          NH3+ -> NH3      -28.4367                  47.7802
[1*]c1cc(C#N)cnc1C#N            NH3+ -> NH3      -23.663                   36.5882
[1*]C(C)(C)CCC1CCCCC1           NH3 -> N2        -14.0888                   9.5008
[1*]C(C)Cc1ccc(Cc2ccccc2)cc1    NH3 -> N2        -13.8458                  13.5031
[1*]C1(CCCCCl)CCCCC1            NH3 -> N2        -11.9534                  11.5473
[1*]C(C)Cc1ccc(CCl)cc1          NH3 -> N2        -11.4337                  17.0659
[1*]c1c(C#N)ccnc1C(=O)Cl        NH3+ -> NH3      -10.6317                  25.9186
[1*]C(C)(C)CC(=C)C              NH3 -> N2         -9.27844                 15.2805
[1*]CCCOc1ccnc2cccnc12          NH3 -> NH3 - N2   -5.68236                 22.2152
[1*]CC=Cc1ncnc2ccccc12          NH3 -> NH3 - N2   -3.85539             